In [ ]:
from scipy.stats import zscore
import numpy as np
import matplotlib.pyplot as plt

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Activation, Permute, Dropout
from tensorflow.keras.layers import Conv2D, MaxPooling2D, AveragePooling2D
from tensorflow.keras.layers import SeparableConv2D, DepthwiseConv2D
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import SpatialDropout2D
from tensorflow.keras.regularizers import l1_l2
from tensorflow.keras.layers import Input, Flatten
from tensorflow.keras.constraints import max_norm
from tensorflow.keras import backend as K

from tensorflow.keras import utils as np_utils
from tensorflow.keras.callbacks import ModelCheckpoint

from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import average_precision_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
from scipy.fft import fft
import pickle

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%cd drive/MyDrive/project

/content/drive/MyDrive/project


In [ ]:
L=8064
Fs=128
Features=np.zeros((25600,32,3*128))
Labels=list()
for subject_id in range(0,32):
    if subject_id % 10 ==0:
      print(subject_id)
    loaded_data=None
    file_name=None
    if subject_id<9:
        file_name= 'data_preprocessed_python/s0'+str(subject_id+1)+'.dat'
    else:
        file_name= 'data_preprocessed_python/s'+str(subject_id+1)+'.dat'
    with open(file_name, 'rb') as f:
        u = pickle._Unpickler(f)
        u.encoding = 'latin1'
        loaded_data = u.load()
    data=loaded_data['data']
    Labels.append(loaded_data['labels'])
    
    for i in range(len(data)):
        for k in range(3,61,3):
          for j in range(32):   
            x=np.array(data[i][j][k*128:(k+3)*128]) -np.array((data[i][j][:3*128]))
            x=zscore(x)
            Features[subject_id*800+i*20+int(k/3-1)][j]=x

0
10
20
30


In [ ]:

def DeepConvNet(nb_classes, Chans = 64, Samples = 256,
                dropoutRate = 0.5):

    # start the model
    input_main   = Input((Chans, Samples, 1))
    block1       = Conv2D(25, (1, 5), 
                                 input_shape=(Chans, Samples, 1),
                                 kernel_constraint = max_norm(2., axis=(0,1,2)))(input_main)
    block1       = Conv2D(25, (Chans, 1),
                                 kernel_constraint = max_norm(2., axis=(0,1,2)))(block1)
    block1       = BatchNormalization(epsilon=1e-05, momentum=0.1)(block1)
    block1       = Activation('elu')(block1)
    block1       = MaxPooling2D(pool_size=(1, 2), strides=(1, 2))(block1)
    block1       = Dropout(dropoutRate)(block1)
  
    block2       = Conv2D(50, (1, 5),
                                 kernel_constraint = max_norm(2., axis=(0,1,2)))(block1)
    block2       = BatchNormalization(epsilon=1e-05, momentum=0.1)(block2)
    block2       = Activation('elu')(block2)
    block2       = MaxPooling2D(pool_size=(1, 2), strides=(1, 2))(block2)
    block2       = Dropout(dropoutRate)(block2)
    
    block3       = Conv2D(100, (1, 5),
                                 kernel_constraint = max_norm(2., axis=(0,1,2)))(block2)
    block3       = BatchNormalization(epsilon=1e-05, momentum=0.1)(block3)
    block3       = Activation('elu')(block3)
    block3       = MaxPooling2D(pool_size=(1, 2), strides=(1, 2))(block3)
    block3       = Dropout(dropoutRate)(block3)
    
    block4       = Conv2D(200, (1, 5),
                                 kernel_constraint = max_norm(2., axis=(0,1,2)))(block3)
    block4       = BatchNormalization(epsilon=1e-05, momentum=0.1)(block4)
    block4       = Activation('elu')(block4)
    block4       = MaxPooling2D(pool_size=(1, 2), strides=(1, 2))(block4)
    block4       = Dropout(dropoutRate)(block4)
    
    flatten      = Flatten()(block4)
    
    dense        = Dense(nb_classes, kernel_constraint = max_norm(0.5))(flatten)
    softmax      = Activation('softmax')(dense)
    
    return Model(inputs=input_main, outputs=softmax)



In [ ]:
y=np.zeros(800*32)
for j in range(32):
  for i in range(40):
    if Labels[j][i][0]>5:
      y[j*800+i*20:j*800+(i+1)*20]=1

In [ ]:
def run(train_index,test_index,
        dropout_rate=0.5):



  Y=np_utils.to_categorical(y)

  d_size, chans, samples = Features.shape
  
  model=DeepConvNet(nb_classes = 2, Chans = chans, Samples = samples, 
               dropoutRate = dropout_rate)

  # compile the model and set the optimizers
  model.compile(loss='categorical_crossentropy', optimizer='adam', 
                metrics = ['accuracy'])
  
  # count number of parameters in the model
  checkpointer = ModelCheckpoint(filepath='/tmp/checkpoint.h5', verbose=1,
                                save_best_only=True)
  
  
  class_weights = {0:np.sum(y)/(len(y)-np.sum(y)), 1:1}
  
  fittedModel = model.fit(Features[train_index], Y[train_index], batch_size = 64, epochs =30 , 
                          verbose=2, validation_data=(Features[test_index], Y[test_index]),
                          callbacks=[checkpointer],class_weight=class_weights)
  probs       = model.predict(Features[test_index])
  preds       = probs.argmax(axis = -1)
  test = Y[test_index].argmax(axis=-1)
  acc = accuracy_score(test, preds)
  cm = confusion_matrix(test, preds)
  f1= f1_score(test, preds)
  return test, preds, acc,f1 , cm, model
  

In [ ]:
kf=KFold(n_splits=5,shuffle=True,random_state=0)

In [ ]:
train_index=list(kf.split(Features))[0][0]
test_index=list(kf.split(Features))[0][1]

In [ ]:
resuslt=run(train_index,test_index)

Epoch 1/30
320/320 - 44s - loss: 0.8883 - accuracy: 0.5301 - val_loss: 0.6655 - val_accuracy: 0.5859

Epoch 00001: val_loss improved from inf to 0.66548, saving model to /tmp/checkpoint.h5
Epoch 2/30
320/320 - 13s - loss: 0.8079 - accuracy: 0.5782 - val_loss: 0.6684 - val_accuracy: 0.5967

Epoch 00002: val_loss did not improve from 0.66548
Epoch 3/30
320/320 - 13s - loss: 0.7623 - accuracy: 0.6065 - val_loss: 0.7033 - val_accuracy: 0.5746

Epoch 00003: val_loss did not improve from 0.66548
Epoch 4/30
320/320 - 13s - loss: 0.7269 - accuracy: 0.6301 - val_loss: 0.7012 - val_accuracy: 0.5814

Epoch 00004: val_loss did not improve from 0.66548
Epoch 5/30
320/320 - 13s - loss: 0.7044 - accuracy: 0.6462 - val_loss: 0.6011 - val_accuracy: 0.6826

Epoch 00005: val_loss improved from 0.66548 to 0.60107, saving model to /tmp/checkpoint.h5
Epoch 6/30
320/320 - 13s - loss: 0.6839 - accuracy: 0.6643 - val_loss: 0.5760 - val_accuracy: 0.6953

Epoch 00006: val_loss improved from 0.60107 to 0.57599, s

In [ ]:
resuslt

(array([1, 1, 1, ..., 1, 1, 1]),
 array([1, 0, 1, ..., 1, 1, 1]),
 0.862109375,
 0.8745557924662403,
 array([[1953,  263],
        [ 443, 2461]]),
 <keras.engine.functional.Functional at 0x7f4270447e90>)

In [ ]:
train_index=list(kf.split(Features))[1][0]
test_index=list(kf.split(Features))[1][1]

In [ ]:
resuslt=run(train_index,test_index)

Epoch 1/30
320/320 - 15s - loss: 0.8859 - accuracy: 0.5323 - val_loss: 0.7321 - val_accuracy: 0.5258

Epoch 00001: val_loss improved from inf to 0.73206, saving model to /tmp/checkpoint.h5
Epoch 2/30
320/320 - 13s - loss: 0.8077 - accuracy: 0.5735 - val_loss: 0.6883 - val_accuracy: 0.5750

Epoch 00002: val_loss improved from 0.73206 to 0.68825, saving model to /tmp/checkpoint.h5
Epoch 3/30
320/320 - 14s - loss: 0.7529 - accuracy: 0.6094 - val_loss: 0.6082 - val_accuracy: 0.6682

Epoch 00003: val_loss improved from 0.68825 to 0.60825, saving model to /tmp/checkpoint.h5
Epoch 4/30
320/320 - 13s - loss: 0.7245 - accuracy: 0.6263 - val_loss: 0.6230 - val_accuracy: 0.6465

Epoch 00004: val_loss did not improve from 0.60825
Epoch 5/30
320/320 - 13s - loss: 0.6987 - accuracy: 0.6534 - val_loss: 0.5992 - val_accuracy: 0.6756

Epoch 00005: val_loss improved from 0.60825 to 0.59917, saving model to /tmp/checkpoint.h5
Epoch 6/30
320/320 - 13s - loss: 0.6796 - accuracy: 0.6690 - val_loss: 0.5587 -

In [ ]:
resuslt

(array([1, 1, 1, ..., 1, 1, 1]),
 array([1, 1, 1, ..., 1, 0, 0]),
 0.8662109375,
 0.8783087582163794,
 array([[1963,  344],
        [ 341, 2472]]),
 <keras.engine.functional.Functional at 0x7f422e3549d0>)

In [ ]:
train_index=list(kf.split(Features))[2][0]
test_index=list(kf.split(Features))[2][1]

In [ ]:
resuslt=run(train_index,test_index)

Epoch 1/30


In [ ]:
resuslt

(array([1, 1, 1, ..., 0, 1, 1]),
 array([1, 0, 1, ..., 0, 1, 1]),
 0.8744140625,
 0.8861342305649018,
 array([[1975,  300],
        [ 343, 2502]]),
 <keras.engine.functional.Functional at 0x7f422e351510>)

In [ ]:
train_index=list(kf.split(Features))[3][0]
test_index=list(kf.split(Features))[3][1]

In [ ]:
resuslt=run(train_index,test_index)

Epoch 1/30
320/320 - 15s - loss: 0.8833 - accuracy: 0.5343 - val_loss: 0.7066 - val_accuracy: 0.5437

Epoch 00001: val_loss improved from inf to 0.70657, saving model to /tmp/checkpoint.h5
Epoch 2/30
320/320 - 13s - loss: 0.8124 - accuracy: 0.5746 - val_loss: 1.0362 - val_accuracy: 0.4633

Epoch 00002: val_loss did not improve from 0.70657
Epoch 3/30
320/320 - 13s - loss: 0.7665 - accuracy: 0.6009 - val_loss: 0.6941 - val_accuracy: 0.5781

Epoch 00003: val_loss improved from 0.70657 to 0.69406, saving model to /tmp/checkpoint.h5
Epoch 4/30
320/320 - 13s - loss: 0.7202 - accuracy: 0.6356 - val_loss: 0.8101 - val_accuracy: 0.5270

Epoch 00004: val_loss did not improve from 0.69406
Epoch 5/30
320/320 - 13s - loss: 0.6960 - accuracy: 0.6546 - val_loss: 0.6541 - val_accuracy: 0.6357

Epoch 00005: val_loss improved from 0.69406 to 0.65406, saving model to /tmp/checkpoint.h5
Epoch 6/30
320/320 - 13s - loss: 0.6701 - accuracy: 0.6706 - val_loss: 0.5427 - val_accuracy: 0.7307

Epoch 00006: val_

In [ ]:
resuslt

(array([1, 1, 1, ..., 1, 1, 1]),
 array([0, 0, 0, ..., 0, 0, 0]),
 0.8603515625,
 0.868008122577072,
 array([[2054,  244],
        [ 471, 2351]]),
 <keras.engine.functional.Functional at 0x7fea65b9e3d0>)

In [ ]:
train_index=list(kf.split(Features))[4][0]
test_index=list(kf.split(Features))[4][1]

In [ ]:
resuslt=run(train_index,test_index)

Epoch 1/30
320/320 - 15s - loss: 0.8963 - accuracy: 0.5305 - val_loss: 0.6489 - val_accuracy: 0.6242

Epoch 00001: val_loss improved from inf to 0.64893, saving model to /tmp/checkpoint.h5
Epoch 2/30
320/320 - 13s - loss: 0.8180 - accuracy: 0.5728 - val_loss: 0.7113 - val_accuracy: 0.5434

Epoch 00002: val_loss did not improve from 0.64893
Epoch 3/30
320/320 - 13s - loss: 0.7667 - accuracy: 0.6000 - val_loss: 0.6888 - val_accuracy: 0.5941

Epoch 00003: val_loss did not improve from 0.64893
Epoch 4/30
320/320 - 13s - loss: 0.7438 - accuracy: 0.6217 - val_loss: 0.5964 - val_accuracy: 0.6564

Epoch 00004: val_loss improved from 0.64893 to 0.59636, saving model to /tmp/checkpoint.h5
Epoch 5/30
320/320 - 13s - loss: 0.7064 - accuracy: 0.6463 - val_loss: 0.5724 - val_accuracy: 0.6998

Epoch 00005: val_loss improved from 0.59636 to 0.57238, saving model to /tmp/checkpoint.h5
Epoch 6/30
320/320 - 13s - loss: 0.6876 - accuracy: 0.6599 - val_loss: 0.6113 - val_accuracy: 0.6582

Epoch 00006: val_

In [ ]:
resuslt

(array([1, 1, 1, ..., 1, 1, 1]),
 array([0, 1, 1, ..., 1, 1, 1]),
 0.8552734375,
 0.8729205968101527,
 array([[1834,  510],
        [ 231, 2545]]),
 <keras.engine.functional.Functional at 0x7feac392a290>)